In [34]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Masking, Dropout, Bidirectional, GRU
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [1]:
import numpy as np
import pandas as pd
import pickle

In [40]:
from sklearn.metrics import classification_report

In [2]:
base_dir = '/Users/vaibhav/MiscProjects/question-classification/'

In [3]:
data_df2_tr = pickle.load(open(base_dir + 'data_df2_tr.pkl', 'rb'))
data_df2_ts = pickle.load(open(base_dir + 'data_df2_ts.pkl', 'rb'))

In [5]:
data_df2_tr.head(3)

,question,category,original,row_id,f_Competition,f_Customer,f_Financials,f_Product,f_Strategy,0,...,18,19,20,21,22,23,24,25,26,27
216,What about returns of capital for comparable s...,Financials,1,216,0,0,1,0,0,0,...,0,0,324,1098,195,576,32,1005,79,368
17,Can you talk about the HBP EBITDA improvement ...,Financials,1,17,0,0,1,0,0,0,...,293,1098,51,279,517,854,459,324,527,51
76,What is G&A expense?,Financials,1,76,0,0,1,0,0,0,...,0,0,0,0,0,0,0,324,342,369


In [6]:
x_tr = data_df2_tr.values[:, 9:]
x_ts = data_df2_ts.values[:, 9:]
y_tr = data_df2_tr.values[:, 4:9]
y_ts = data_df2_ts.values[:, 4:9]

In [7]:
print(x_tr.shape)
print(x_ts.shape)
print(y_tr.shape)
print(y_ts.shape)

(401, 28)
(125, 28)
(401, 5)
(125, 5)


In [4]:
embedding_matrix = pickle.load(open(base_dir + 'embedding_matrix.pkl', 'rb'))

In [20]:
time_steps = 28
onehot_vec_size = 50
ncats = 5

lstm_size1 = 16

input1 = Input(shape=(time_steps,), dtype='int32', name = 'input')

# embedding layer to convert into one-hot encoded vector
# 0 is mapped to all zeros - this will be ignored when masked
# hence, 1st row of embedding matrix is all zero
# rest of the matrix is just an identity matrix
# this matrix is marked as non-trainable 

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=time_steps,
                            trainable=False)

embedded_sequences1 = embedding_layer(input1)

lstm1 = LSTM(lstm_size1, dropout=0.3, recurrent_dropout=0.3)

y1 = Masking(mask_value=0.0)(embedded_sequences1)

y1 = lstm1(y1)
y = Dropout(0.3)(y1)
y = Dense(ncats, activation='softmax')(y)

model1 = Model(inputs = input1, outputs = y)

In [21]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [18]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 28, 50)            55500     
_________________________________________________________________
masking_2 (Masking)          (None, 28, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                4288      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 85        
Total params: 59,873
Trainable params: 4,373
Non-trainable params: 55,500
_________________________________________________________________


In [22]:
model1.fit(x_tr, y_tr,
          batch_size=32,
          epochs=100,
          validation_data=(x_ts, y_ts),
          #callbacks=callbacks,
          verbose=2,
         )

Train on 401 samples, validate on 125 samples
Epoch 1/100
 - 2s - loss: 1.6205 - acc: 0.2369 - val_loss: 1.5945 - val_acc: 0.2400
Epoch 2/100
 - 0s - loss: 1.6155 - acc: 0.2244 - val_loss: 1.5811 - val_acc: 0.2640
Epoch 3/100
 - 0s - loss: 1.6019 - acc: 0.2444 - val_loss: 1.5697 - val_acc: 0.2800
Epoch 4/100
 - 0s - loss: 1.5963 - acc: 0.1895 - val_loss: 1.5592 - val_acc: 0.3120
Epoch 5/100
 - 0s - loss: 1.6056 - acc: 0.2444 - val_loss: 1.5497 - val_acc: 0.3440
Epoch 6/100
 - 0s - loss: 1.5653 - acc: 0.3117 - val_loss: 1.5402 - val_acc: 0.3680
Epoch 7/100
 - 0s - loss: 1.5687 - acc: 0.2793 - val_loss: 1.5309 - val_acc: 0.3680
Epoch 8/100
 - 0s - loss: 1.5590 - acc: 0.2868 - val_loss: 1.5202 - val_acc: 0.4080
Epoch 9/100
 - 0s - loss: 1.5419 - acc: 0.2743 - val_loss: 1.5102 - val_acc: 0.4000
Epoch 10/100
 - 0s - loss: 1.5377 - acc: 0.3192 - val_loss: 1.5003 - val_acc: 0.4160
Epoch 11/100
 - 0s - loss: 1.5141 - acc: 0.3691 - val_loss: 1.4892 - val_acc: 0.4080
Epoch 12/100
 - 1s - loss: 1

In [30]:
lstm_size1 = 16

input1 = Input(shape=(time_steps,), dtype='int32', name = 'input')

# embedding layer to convert into one-hot encoded vector
# 0 is mapped to all zeros - this will be ignored when masked
# hence, 1st row of embedding matrix is all zero
# rest of the matrix is just an identity matrix
# this matrix is marked as non-trainable 

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=time_steps,
                            trainable=False)

embedded_sequences1 = embedding_layer(input1)

bilstm1 = Bidirectional(LSTM(lstm_size1, dropout=0.3, recurrent_dropout=0.3, return_sequences=False))

y1 = Masking(mask_value=0.0)(embedded_sequences1)

y1 = bilstm1(y1)
y = Dropout(0.3)(y1)
y = Dense(ncats, activation='softmax')(y)

model2 = Model(inputs = input1, outputs = y)

In [32]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [33]:
model2.fit(x_tr, y_tr,
          batch_size=32,
          epochs=100,
          validation_data=(x_ts, y_ts),
          #callbacks=callbacks,
          verbose=2,
         )

Train on 401 samples, validate on 125 samples
Epoch 1/100
 - 3s - loss: 1.6927 - acc: 0.1920 - val_loss: 1.6085 - val_acc: 0.2480
Epoch 2/100
 - 0s - loss: 1.6816 - acc: 0.2195 - val_loss: 1.5870 - val_acc: 0.2320
Epoch 3/100
 - 0s - loss: 1.6654 - acc: 0.1920 - val_loss: 1.5767 - val_acc: 0.2800
Epoch 4/100
 - 0s - loss: 1.6348 - acc: 0.1970 - val_loss: 1.5683 - val_acc: 0.3360
Epoch 5/100
 - 0s - loss: 1.6173 - acc: 0.2444 - val_loss: 1.5600 - val_acc: 0.3440
Epoch 6/100
 - 0s - loss: 1.5928 - acc: 0.2718 - val_loss: 1.5519 - val_acc: 0.3600
Epoch 7/100
 - 1s - loss: 1.5934 - acc: 0.2594 - val_loss: 1.5444 - val_acc: 0.4080
Epoch 8/100
 - 1s - loss: 1.5663 - acc: 0.2968 - val_loss: 1.5368 - val_acc: 0.3680
Epoch 9/100
 - 1s - loss: 1.5512 - acc: 0.3242 - val_loss: 1.5261 - val_acc: 0.4480
Epoch 10/100
 - 1s - loss: 1.5528 - acc: 0.3267 - val_loss: 1.5153 - val_acc: 0.4560
Epoch 11/100
 - 1s - loss: 1.5404 - acc: 0.3192 - val_loss: 1.5052 - val_acc: 0.4320
Epoch 12/100
 - 0s - loss: 1

In [35]:
gru_size1 = 16

input1 = Input(shape=(time_steps,), dtype='int32', name = 'input')

# embedding layer to convert into one-hot encoded vector
# 0 is mapped to all zeros - this will be ignored when masked
# hence, 1st row of embedding matrix is all zero
# rest of the matrix is just an identity matrix
# this matrix is marked as non-trainable 

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=time_steps,
                            trainable=False)

embedded_sequences1 = embedding_layer(input1)

bigru1 = Bidirectional(GRU(gru_size1, dropout=0.3, recurrent_dropout=0.3, return_sequences=False))

y1 = Masking(mask_value=0.0)(embedded_sequences1)

y1 = bigru1(y1)
y = Dropout(0.3)(y1)
y = Dense(ncats, activation='softmax')(y)

model3 = Model(inputs = input1, outputs = y)

In [36]:
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [37]:
model3.fit(x_tr, y_tr,
          batch_size=32,
          epochs=100,
          validation_data=(x_ts, y_ts),
          #callbacks=callbacks,
          verbose=2,
         )

Train on 401 samples, validate on 125 samples
Epoch 1/100
 - 3s - loss: 1.7851 - acc: 0.2294 - val_loss: 1.6772 - val_acc: 0.2240
Epoch 2/100
 - 0s - loss: 1.6752 - acc: 0.2120 - val_loss: 1.6195 - val_acc: 0.1840
Epoch 3/100
 - 0s - loss: 1.6902 - acc: 0.2070 - val_loss: 1.5972 - val_acc: 0.2240
Epoch 4/100
 - 0s - loss: 1.6463 - acc: 0.2594 - val_loss: 1.5798 - val_acc: 0.2640
Epoch 5/100
 - 0s - loss: 1.6191 - acc: 0.2544 - val_loss: 1.5650 - val_acc: 0.2880
Epoch 6/100
 - 0s - loss: 1.6190 - acc: 0.2594 - val_loss: 1.5504 - val_acc: 0.2720
Epoch 7/100
 - 0s - loss: 1.5806 - acc: 0.2718 - val_loss: 1.5374 - val_acc: 0.2800
Epoch 8/100
 - 0s - loss: 1.5475 - acc: 0.2768 - val_loss: 1.5228 - val_acc: 0.3040
Epoch 9/100
 - 0s - loss: 1.5224 - acc: 0.3491 - val_loss: 1.5099 - val_acc: 0.3280
Epoch 10/100
 - 0s - loss: 1.5169 - acc: 0.3566 - val_loss: 1.5010 - val_acc: 0.3200
Epoch 11/100
 - 0s - loss: 1.5163 - acc: 0.3267 - val_loss: 1.4866 - val_acc: 0.3680
Epoch 12/100
 - 0s - loss: 1

In [41]:
y_pred = model3.predict(x_ts)

In [47]:
pd.DataFrame(classification_report(np.argmax(y_ts, axis=1), np.argmax(y_pred, axis=1), 
             output_dict=True)).transpose()

,f1-score,precision,recall,support
0,0.583333,0.608696,0.560,25.0
1,0.680851,0.727273,0.640,25.0
2,0.666667,0.750000,0.600,25.0
3,0.489796,0.500000,0.480,25.0
4,0.491803,0.416667,0.600,25.0
micro avg,0.576000,0.576000,0.576,125.0
macro avg,0.582490,0.600527,0.576,125.0
weighted avg,0.582490,0.600527,0.576,125.0


In [48]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix],
                  trainable=False)(inp)
    x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.25,
                           recurrent_dropout=0.25))(x)
    x = Attention(maxlen)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)

In [49]:
gru_size1 = 16

input1 = Input(shape=(time_steps,), dtype='int32', name = 'input')

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=time_steps,
                            trainable=False)

embedded_sequences1 = embedding_layer(input1)

bigru1 = Bidirectional(GRU(gru_size1, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))

y1 = Masking(mask_value=0.0)(embedded_sequences1)

y1 = bigru1(y1)

y1 = Attention(time_steps)(y1)

y = Dropout(0.3)(y1)
y = Dense(ncats, activation='softmax')(y)

model4 = Model(inputs = input1, outputs = y)

In [50]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [51]:
model4.fit(x_tr, y_tr,
          batch_size=32,
          epochs=100,
          validation_data=(x_ts, y_ts),
          #callbacks=callbacks,
          verbose=2,
         )

Train on 401 samples, validate on 125 samples
Epoch 1/100
 - 3s - loss: 1.7708 - acc: 0.2195 - val_loss: 1.6188 - val_acc: 0.2400
Epoch 2/100
 - 0s - loss: 1.6925 - acc: 0.2095 - val_loss: 1.5918 - val_acc: 0.2640
Epoch 3/100
 - 0s - loss: 1.6752 - acc: 0.2269 - val_loss: 1.5776 - val_acc: 0.2560
Epoch 4/100
 - 0s - loss: 1.6524 - acc: 0.2369 - val_loss: 1.5656 - val_acc: 0.3120
Epoch 5/100
 - 0s - loss: 1.6219 - acc: 0.2319 - val_loss: 1.5528 - val_acc: 0.2960
Epoch 6/100
 - 0s - loss: 1.6028 - acc: 0.2519 - val_loss: 1.5411 - val_acc: 0.3200
Epoch 7/100
 - 0s - loss: 1.6146 - acc: 0.2319 - val_loss: 1.5281 - val_acc: 0.3440
Epoch 8/100
 - 0s - loss: 1.5698 - acc: 0.2693 - val_loss: 1.5181 - val_acc: 0.3520
Epoch 9/100
 - 0s - loss: 1.5434 - acc: 0.3142 - val_loss: 1.5046 - val_acc: 0.4160
Epoch 10/100
 - 0s - loss: 1.5423 - acc: 0.3017 - val_loss: 1.4902 - val_acc: 0.4000
Epoch 11/100
 - 0s - loss: 1.5134 - acc: 0.3516 - val_loss: 1.4760 - val_acc: 0.4160
Epoch 12/100
 - 0s - loss: 1

In [52]:
data_df2_tr2 = pickle.load(open(base_dir + 'data_df2_tr2.pkl', 'rb'))
data_df2_ts2 = pickle.load(open(base_dir + 'data_df2_ts2.pkl', 'rb'))
elmo_embeddings_matrix = pickle.load(open(base_dir + 'elmo_embeddings_matrix.pkl', 'rb'))

In [53]:
elmo_embeddings_matrix.shape

(6026, 1024)

In [55]:
data_df2_tr.shape

(401, 37)

In [57]:
x_tr2 = data_df2_tr2.values[:, 37:]
x_ts2 = data_df2_ts2.values[:, 37:]
y_tr2 = data_df2_tr2.values[:, 4:9]
y_ts2 = data_df2_ts2.values[:, 4:9]

In [58]:
print(x_tr2.shape)
print(x_ts2.shape)
print(y_tr2.shape)
print(y_ts2.shape)

(401, 38)
(125, 38)
(401, 5)
(125, 5)


In [75]:
gru_size1 = 4
time_steps = 38

input1 = Input(shape=(time_steps,), dtype='int32', name = 'input')

embedding_layer = Embedding(elmo_embeddings_matrix.shape[0],
                            elmo_embeddings_matrix.shape[1],
                            weights=[elmo_embeddings_matrix],
                            input_length=time_steps,
                            trainable=False)

embedded_sequences1 = embedding_layer(input1)

bigru1 = Bidirectional(GRU(gru_size1, dropout=0.4, recurrent_dropout=0.3, return_sequences=True))

y1 = Masking(mask_value=0.0)(embedded_sequences1)

y1 = bigru1(y1)

y1 = Attention(time_steps)(y1)

y = Dropout(0.5)(y1)
y = Dense(ncats, activation='softmax')(y)

model5 = Model(inputs = input1, outputs = y)

In [76]:
model5.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [77]:
model5.fit(x_tr2, y_tr2,
          batch_size=32,
          epochs=20,
          validation_data=(x_ts2, y_ts2),
          #callbacks=callbacks,
          verbose=2,
         )

Train on 401 samples, validate on 125 samples
Epoch 1/20
 - 6s - loss: 1.7057 - acc: 0.2344 - val_loss: 1.5664 - val_acc: 0.2800
Epoch 2/20
 - 1s - loss: 1.5983 - acc: 0.2793 - val_loss: 1.5113 - val_acc: 0.3360
Epoch 3/20
 - 1s - loss: 1.5193 - acc: 0.3167 - val_loss: 1.4739 - val_acc: 0.3680
Epoch 4/20
 - 1s - loss: 1.4560 - acc: 0.3940 - val_loss: 1.4457 - val_acc: 0.4000
Epoch 5/20
 - 1s - loss: 1.3713 - acc: 0.4414 - val_loss: 1.3957 - val_acc: 0.4880
Epoch 6/20
 - 1s - loss: 1.3410 - acc: 0.4489 - val_loss: 1.3737 - val_acc: 0.4560
Epoch 7/20
 - 1s - loss: 1.3021 - acc: 0.4988 - val_loss: 1.3238 - val_acc: 0.5120
Epoch 8/20
 - 1s - loss: 1.2791 - acc: 0.4913 - val_loss: 1.2967 - val_acc: 0.5440
Epoch 9/20
 - 1s - loss: 1.1922 - acc: 0.5561 - val_loss: 1.2921 - val_acc: 0.5600
Epoch 10/20
 - 1s - loss: 1.1791 - acc: 0.5810 - val_loss: 1.2646 - val_acc: 0.5520
Epoch 11/20
 - 1s - loss: 1.1234 - acc: 0.5935 - val_loss: 1.2605 - val_acc: 0.4880
Epoch 12/20
 - 1s - loss: 1.0965 - acc:

In [73]:
y_pred5 = model5.predict(x_ts2)

In [74]:
pd.DataFrame(classification_report(np.argmax(y_ts2, axis=1), np.argmax(y_pred5, axis=1), 
             output_dict=True)).transpose()

,f1-score,precision,recall,support
0,0.716981,0.678571,0.76,25.0
1,0.711111,0.800000,0.64,25.0
2,0.692308,0.666667,0.72,25.0
3,0.461538,0.444444,0.48,25.0
4,0.625000,0.652174,0.60,25.0
micro avg,0.640000,0.640000,0.64,125.0
macro avg,0.641388,0.648371,0.64,125.0
weighted avg,0.641388,0.648371,0.64,125.0
